# Updation of previous model with Adaptive Moment Estimation (Adam) optimizer

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam # Import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau # Import ReduceLROnPlateau
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# MobileNetV2 is commonly trained on 224x224 images
height = 224
width = 224

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(height, width, 3))

In [6]:
# --- Data Loading and Augmentation ---
train_dir = "dataset/training"
validation_dir = "dataset/validation"
batch_size = 8

In [7]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input, # Use MobileNetV2's preprocess_input
    rotation_range=90,
    horizontal_flip=True,
    vertical_flip=True
)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(height, width),
    batch_size=batch_size
)

Found 2774 images belonging to 2 classes.


In [8]:
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(height, width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 592 images belonging to 2 classes.


In [9]:
def build_finetune_model(base_model, dropout, fc_layers, num_classes):
    # Freeze the layers of the base model
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom top layers for classification
    x = base_model.output
    x = Flatten()(x)
    for fc in fc_layers:
        x = Dense(fc, activation='relu')(x)
        x = Dropout(dropout)(x)

    predictions = Dense(num_classes, activation='softmax')(x)

    finetune_model = Model(inputs=base_model.input, outputs=predictions)
    return finetune_model

In [10]:
class_list = ['Real', 'Fake']
FC_Layers = [1024, 1024]
dropout = 0.5

finetune_model = build_finetune_model(
    base_model,
    dropout=dropout,
    fc_layers=FC_Layers,
    num_classes=len(class_list)
)

In [11]:
num_epochs = 20
num_train_images = 2774
num_validation_images = 592

In [12]:
# Callbacks for saving the best model and stopping early
checkpoint = ModelCheckpoint(
    "MobileNetV2_model_improved.h5", # Using a new name for the improved model
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch'
)
early = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0,
    patience=10, # Can be reduced as the scheduler will help
    verbose=1,
    mode="auto"
)

# We are using ReduceLROnPlateau, which will reduce the learning rate when a metric (validation_accuracy) has stopped improving.

In [13]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5,
    verbose=1,
    min_lr=0.00001
)

# Compile the model with the Adam optimizer
optimizer = Adam(learning_rate=0.001)
finetune_model.compile(
    loss="categorical_crossentropy",
    optimizer=optimizer,
    metrics=['accuracy']
)

callbacks = [checkpoint, early, reduce_lr]

In [14]:
finetune_model.fit(
    train_generator,
    steps_per_epoch=num_train_images // batch_size,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=num_validation_images // batch_size, # Corrected validation_steps
    callbacks=callbacks # Use the new callbacks list
)

c:\Users\PRATYAOY SARKAR\FinalYrProject\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6607 - loss: 19.3735
Epoch 1: val_accuracy improved from None to 0.68074, saving model to MobileNetV2_model_improved.h5


346/346 ━━━━━━━━━━━━━━━━━━━━ 437s 1s/step - accuracy: 0.7061 - loss: 10.4164 - val_accuracy: 0.6807 - val_loss: 0.8353 - learning_rate: 0.0010
Epoch 2/20
  1/346 ━━━━━━━━━━━━━━━━━━━━ 6:30 1s/step - accuracy: 0.5000 - loss: 0.8468

c:\Users\PRATYAOY SARKAR\FinalYrProject\myenv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 2: val_accuracy did not improve from 0.68074
346/346 ━━━━━━━━━━━━━━━━━━━━ 20s 55ms/step - accuracy: 0.5000 - loss: 0.8468 - val_accuracy: 0.6723 - val_loss: 0.7964 - learning_rate: 0.0010
Epoch 3/20
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7351 - loss: 0.7932
Epoch 3: val_accuracy improved from 0.68074 to 0.75676, saving model to MobileNetV2_model_improved.h5


346/346 ━━━━━━━━━━━━━━━━━━━━ 403s 1s/step - accuracy: 0.7339 - loss: 0.7082 - val_accuracy: 0.7568 - val_loss: 0.5253 - learning_rate: 0.0010
Epoch 4/20
  1/346 ━━━━━━━━━━━━━━━━━━━━ 6:26 1s/step - accuracy: 0.8750 - loss: 0.3415
Epoch 4: val_accuracy did not improve from 0.75676
346/346 ━━━━━━━━━━━━━━━━━━━━ 19s 53ms/step - accuracy: 0.8750 - loss: 0.3415 - val_accuracy: 0.7534 - val_loss: 0.5254 - learning_rate: 0.0010
Epoch 5/20
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7503 - loss: 0.6399
Epoch 5: val_accuracy improved from 0.75676 to 0.76182, saving model to MobileNetV2_model_improved.h5


346/346 ━━━━━━━━━━━━━━━━━━━━ 407s 1s/step - accuracy: 0.7393 - loss: 0.6390 - val_accuracy: 0.7618 - val_loss: 0.6166 - learning_rate: 0.0010
Epoch 6/20
  1/346 ━━━━━━━━━━━━━━━━━━━━ 7:18 1s/step - accuracy: 0.7500 - loss: 0.2781
Epoch 6: val_accuracy improved from 0.76182 to 0.79223, saving model to MobileNetV2_model_improved.h5


346/346 ━━━━━━━━━━━━━━━━━━━━ 23s 63ms/step - accuracy: 0.7500 - loss: 0.2781 - val_accuracy: 0.7922 - val_loss: 0.5454 - learning_rate: 0.0010
Epoch 7/20
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7178 - loss: 0.8049
Epoch 7: val_accuracy did not improve from 0.79223
346/346 ━━━━━━━━━━━━━━━━━━━━ 464s 1s/step - accuracy: 0.7209 - loss: 0.6881 - val_accuracy: 0.7382 - val_loss: 0.5866 - learning_rate: 0.0010
Epoch 8/20
  1/346 ━━━━━━━━━━━━━━━━━━━━ 7:35 1s/step - accuracy: 0.8750 - loss: 0.2385
Epoch 8: val_accuracy did not improve from 0.79223

Epoch 8: ReduceLROnPlateau reducing learning rate to 0.00020000000949949026.
346/346 ━━━━━━━━━━━━━━━━━━━━ 24s 65ms/step - accuracy: 0.8750 - loss: 0.2385 - val_accuracy: 0.7365 - val_loss: 0.5829 - learning_rate: 0.0010
Epoch 9/20
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7410 - loss: 0.5165
Epoch 9: val_accuracy did not improve from 0.79223
346/346 ━━━━━━━━━━━━━━━━━━━━ 408s 1s/step - accuracy: 0.7621 - loss: 0.4773 - val_ac

346/346 ━━━━━━━━━━━━━━━━━━━━ 21s 57ms/step - accuracy: 0.7500 - loss: 0.2788 - val_accuracy: 0.7956 - val_loss: 0.4997 - learning_rate: 2.0000e-04
Epoch 15/20
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.7969 - loss: 0.4066
Epoch 15: val_accuracy improved from 0.79561 to 0.81926, saving model to MobileNetV2_model_improved.h5


346/346 ━━━━━━━━━━━━━━━━━━━━ 397s 1s/step - accuracy: 0.8069 - loss: 0.4043 - val_accuracy: 0.8193 - val_loss: 0.4651 - learning_rate: 2.0000e-04
Epoch 16/20
  1/346 ━━━━━━━━━━━━━━━━━━━━ 6:11 1s/step - accuracy: 1.0000 - loss: 0.3438
Epoch 16: val_accuracy did not improve from 0.81926
346/346 ━━━━━━━━━━━━━━━━━━━━ 19s 52ms/step - accuracy: 1.0000 - loss: 0.3438 - val_accuracy: 0.8193 - val_loss: 0.4676 - learning_rate: 2.0000e-04
Epoch 17/20
346/346 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.8267 - loss: 0.3698
Epoch 17: val_accuracy did not improve from 0.81926
346/346 ━━━━━━━━━━━━━━━━━━━━ 390s 1s/step - accuracy: 0.8196 - loss: 0.3810 - val_accuracy: 0.7770 - val_loss: 0.4786 - learning_rate: 2.0000e-04
Epoch 18/20
  1/346 ━━━━━━━━━━━━━━━━━━━━ 6:04 1s/step - accuracy: 0.8750 - loss: 0.4454
Epoch 18: val_accuracy did not improve from 0.81926
346/346 ━━━━━━━━━━━━━━━━━━━━ 19s 52ms/step - accuracy: 0.8750 - loss: 0.4454 - val_accuracy: 0.7703 - val_loss: 0.4795 - learning_rate: 2.0000e

In [15]:
MODEL_PATH = "MobileNetV2_model_improved.h5" 

model = load_model(MODEL_PATH)
true_labels = validation_generator.classes
class_labels = list(validation_generator.class_indices.keys())

predictions = model.predict(validation_generator)
predicted_labels = np.argmax(predictions, axis=1)


74/74 ━━━━━━━━━━━━━━━━━━━━ 20s 250ms/step


In [16]:
from sklearn.metrics import classification_report, accuracy_score

In [17]:
print("--- Model Evaluation Report ---")
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"Accuracy: {accuracy:.4f}\n")

# Print precision, recall, f1-score for each class
print("Classification Report:")
print(classification_report(true_labels, predicted_labels, target_names=class_labels))

--- Model Evaluation Report ---
Accuracy: 0.8193

Classification Report:
              precision    recall  f1-score   support

        fake       0.78      0.89      0.83       296
        real       0.87      0.75      0.81       296

    accuracy                           0.82       592
   macro avg       0.83      0.82      0.82       592
weighted avg       0.83      0.82      0.82       592

